In [1]:
import time
import argparse
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import torch
import numpy as np
import cv2
from skimage.measure import compare_ssim, compare_psnr

import matplotlib.pyplot as plt
import utils
import dataset

def str2bool(v):
    #print(v)
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Unsupported value encountered.')

if __name__ == "__main__":
    # ----------------------------------------
    #        Initialize the parameters
    # ----------------------------------------
    parser = argparse.ArgumentParser()
    #GPU parameters
    parser.add_argument('--no_gpu', default = False, help = 'True for CPU')
    # Saving, and loading parameters
    
    #TinyDerainGAN_RID
    parser.add_argument('--save_name', type = str, default = '../results/RID/TinyDerainGAN_RID', help = 'save the generated with certain epoch')
    parser.add_argument('--load_gname', type = str, default = '../models/models_TinyDerainGAN_raindrop/KPN_rainy_image_epoch200_bs4_generator.pth', help = 'load the pre-trained model with certain epoch')
    
    parser.add_argument('--baseroot', type = str, default = '../rainy_image_dataset/RID512/images', help = 'images baseroot')
    
    parser.add_argument('--test_batch_size', type = int, default = 1, help = 'size of the batches')
    parser.add_argument('--num_workers', type = int, default = 1, help = 'number of workers')
    # Initialization parameters
    parser.add_argument('--init_type', type = str, default = 'xavier', help = 'initialization type of generator')
    parser.add_argument('--init_gain', type = float, default = 0.02, help = 'initialization gain of generator')
    # Dataset parameters
    parser.add_argument('--crop', type = str2bool, default = False, help = 'whether to crop input images')
    parser.add_argument('--crop_size', type = int, default = 512, help = 'single patch size')
    parser.add_argument('--resize', type = str2bool, default = False, help = 'whether to resize input images')
    parser.add_argument('--scale_size', type = int, default = 512, help = 'single patch size')
    parser.add_argument('--geometry_aug', type = str2bool, default = False, help = 'geometry augmentation (scaling)')
    parser.add_argument('--angle_aug', type = str2bool, default = False, help = 'geometry augmentation (rotation, flipping)')
    parser.add_argument('--scale_min', type = float, default = 1, help = 'min scaling factor')
    parser.add_argument('--scale_max', type = float, default = 1, help = 'max scaling factor')
    parser.add_argument('--add_noise', type = str2bool, default = False, help = 'whether to add noise to input images')
    parser.add_argument('--mu', type = int, default = 0, help = 'Gaussian noise mean')
    parser.add_argument('--sigma', type = int, default = 30, help = 'Gaussian noise variance: 30 | 50 | 70')
    opt = parser.parse_args(args=[])
    print(opt)

    # ----------------------------------------
    #                   Test
    # ----------------------------------------
    # Initialize
    if opt.no_gpu:
        generator = utils.create_generator(opt)
    else:
        generator = utils.create_generator(opt).cuda()

    test_dataset = dataset.DenoisingValDataset(opt)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = opt.test_batch_size, shuffle = False, num_workers = opt.num_workers, pin_memory = True)
    sample_folder = opt.save_name
    utils.check_path(sample_folder)

    psnr_sum, psnr_ave, ssim_sum, ssim_ave, time_sum, time_ave, eval_cnt, iter_psnr, iter_ssim = 0, 0, 0, 0, 0, 0, 0, 0,0
    #np.set_printoptions(threshold=np.inf)
    
    # Count start time
    prev_time = time.time()
    
    # forward
    for i, (true_input, true_target, height_origin, width_origin, file_name) in enumerate(test_loader):
        prev_time = time.time()

        # To device
        if opt.no_gpu:
            true_input = true_input
            true_target = true_target
        else:
            true_input = true_input.cuda()
            true_target = true_target.cuda()            

        # Forward propagation
        with torch.no_grad():
            fake_target, feature_map = generator(true_input, true_input)
        
        time_iter = time.time() - prev_time
        time_sum = time_sum + time_iter

        # Save
        #img_list = [fake_target]
        #name_list = ['pred']
        #sample_name = file_name
        #utils.save_RID_png(sample_folder = sample_folder, sample_name = sample_name, img_list = img_list, name_list = name_list, pixel_max_cnt = 255, height = height_origin, width = width_origin)

        eval_cnt = eval_cnt + 1
        print('The '+ str(i) +'-th iter: sec: ' + str(round(time_iter,6)))
        
    psnr_ave = psnr_sum / eval_cnt
    ssim_ave = ssim_sum / eval_cnt
    time_ave = time_sum / eval_cnt
    fps = 1/time_ave
    eval_file = "../data/evaluation_data.txt"
    
    psnr_content = opt.load_gname + ":  fps: " + str(round(fps)) + "("+ str(round(time_ave,6)) +")\n"
    utils.text_save(content = psnr_content, filename = eval_file)
    
    
    


Namespace(add_noise=False, angle_aug=False, baseroot='../rainy_image_dataset/RID512/images', crop=False, crop_size=512, geometry_aug=False, init_gain=0.02, init_type='xavier', load_gname='../models/models_TinyDerainGAN_raindrop/KPN_rainy_image_epoch200_bs4_generator.pth', mu=0, no_gpu=False, num_workers=1, resize=False, save_name='../results/RID/TinyDerainGAN_RID', scale_max=1, scale_min=1, scale_size=512, sigma=30, test_batch_size=1)
Generator are loaded!
The 0-th iter: sec: 0.065796
The 1-th iter: sec: 0.059007
The 2-th iter: sec: 0.084525
The 3-th iter: sec: 0.077843
The 4-th iter: sec: 0.079957
The 5-th iter: sec: 0.079706
The 6-th iter: sec: 0.078607
The 7-th iter: sec: 0.079691
The 8-th iter: sec: 0.078387
The 9-th iter: sec: 0.079532
The 10-th iter: sec: 0.080878
The 11-th iter: sec: 0.098894
The 12-th iter: sec: 0.079134
The 13-th iter: sec: 0.078703
The 14-th iter: sec: 0.078906
The 15-th iter: sec: 0.079157
The 16-th iter: sec: 0.078726
The 17-th iter: sec: 0.079387
The 18-th

The 255-th iter: sec: 0.052575
The 256-th iter: sec: 0.051565
The 257-th iter: sec: 0.052025
The 258-th iter: sec: 0.051833
The 259-th iter: sec: 0.052803
The 260-th iter: sec: 0.051166
The 261-th iter: sec: 0.051951
The 262-th iter: sec: 0.052144
The 263-th iter: sec: 0.052937
The 264-th iter: sec: 0.050835
The 265-th iter: sec: 0.052098
The 266-th iter: sec: 0.052283
The 267-th iter: sec: 0.048186
The 268-th iter: sec: 0.050748
The 269-th iter: sec: 0.052273
The 270-th iter: sec: 0.052051
The 271-th iter: sec: 0.052876
The 272-th iter: sec: 0.051052
The 273-th iter: sec: 0.051773
The 274-th iter: sec: 0.052078
The 275-th iter: sec: 0.052926
The 276-th iter: sec: 0.050962
The 277-th iter: sec: 0.05249
The 278-th iter: sec: 0.051858
The 279-th iter: sec: 0.05279
The 280-th iter: sec: 0.051603
The 281-th iter: sec: 0.051599
The 282-th iter: sec: 0.052118
The 283-th iter: sec: 0.052576
The 284-th iter: sec: 0.050986
The 285-th iter: sec: 0.052204
The 286-th iter: sec: 0.051461
The 287-th

The 524-th iter: sec: 0.052799
The 525-th iter: sec: 0.052047
The 526-th iter: sec: 0.052353
The 527-th iter: sec: 0.052356
The 528-th iter: sec: 0.052658
The 529-th iter: sec: 0.052406
The 530-th iter: sec: 0.052119
The 531-th iter: sec: 0.052459
The 532-th iter: sec: 0.052862
The 533-th iter: sec: 0.052084
The 534-th iter: sec: 0.052778
The 535-th iter: sec: 0.052426
The 536-th iter: sec: 0.052953
The 537-th iter: sec: 0.052803
The 538-th iter: sec: 0.051881
The 539-th iter: sec: 0.052885
The 540-th iter: sec: 0.052338
The 541-th iter: sec: 0.052645
The 542-th iter: sec: 0.052377
The 543-th iter: sec: 0.05245
The 544-th iter: sec: 0.052866
The 545-th iter: sec: 0.052198
The 546-th iter: sec: 0.052046
The 547-th iter: sec: 0.052537
The 548-th iter: sec: 0.053125
The 549-th iter: sec: 0.051848
The 550-th iter: sec: 0.052969
The 551-th iter: sec: 0.052005
The 552-th iter: sec: 0.052653
The 553-th iter: sec: 0.052359
The 554-th iter: sec: 0.052329
The 555-th iter: sec: 0.052779
The 556-t

The 792-th iter: sec: 0.053048
The 793-th iter: sec: 0.05307
The 794-th iter: sec: 0.052746
The 795-th iter: sec: 0.052917
The 796-th iter: sec: 0.052686
The 797-th iter: sec: 0.053029
The 798-th iter: sec: 0.052908
The 799-th iter: sec: 0.052301
The 800-th iter: sec: 0.053497
The 801-th iter: sec: 0.052339
The 802-th iter: sec: 0.05312
The 803-th iter: sec: 0.053014
The 804-th iter: sec: 0.053205
The 805-th iter: sec: 0.052826
The 806-th iter: sec: 0.052862
The 807-th iter: sec: 0.053068
The 808-th iter: sec: 0.053204
The 809-th iter: sec: 0.052897
The 810-th iter: sec: 0.053056
The 811-th iter: sec: 0.052721
The 812-th iter: sec: 0.053326
The 813-th iter: sec: 0.052751
The 814-th iter: sec: 0.052913
The 815-th iter: sec: 0.053362
The 816-th iter: sec: 0.053399
The 817-th iter: sec: 0.053143
The 818-th iter: sec: 0.052749
The 819-th iter: sec: 0.052991
The 820-th iter: sec: 0.053309
The 821-th iter: sec: 0.053097
The 822-th iter: sec: 0.052636
The 823-th iter: sec: 0.053166
The 824-th

The 1056-th iter: sec: 0.053273
The 1057-th iter: sec: 0.05412
The 1058-th iter: sec: 0.052722
The 1059-th iter: sec: 0.053031
The 1060-th iter: sec: 0.053574
The 1061-th iter: sec: 0.052891
The 1062-th iter: sec: 0.053558
The 1063-th iter: sec: 0.05303
The 1064-th iter: sec: 0.053128
The 1065-th iter: sec: 0.053387
The 1066-th iter: sec: 0.053183
The 1067-th iter: sec: 0.053279
The 1068-th iter: sec: 0.053889
The 1069-th iter: sec: 0.053444
The 1070-th iter: sec: 0.053447
The 1071-th iter: sec: 0.052683
The 1072-th iter: sec: 0.053264
The 1073-th iter: sec: 0.053488
The 1074-th iter: sec: 0.052777
The 1075-th iter: sec: 0.053512
The 1076-th iter: sec: 0.053537
The 1077-th iter: sec: 0.052857
The 1078-th iter: sec: 0.053385
The 1079-th iter: sec: 0.053509
The 1080-th iter: sec: 0.053162
The 1081-th iter: sec: 0.053451
The 1082-th iter: sec: 0.053193
The 1083-th iter: sec: 0.053046
The 1084-th iter: sec: 0.053808
The 1085-th iter: sec: 0.052903
The 1086-th iter: sec: 0.053812
The 1087-t

The 1316-th iter: sec: 0.053321
The 1317-th iter: sec: 0.053669
The 1318-th iter: sec: 0.054005
The 1319-th iter: sec: 0.053349
The 1320-th iter: sec: 0.053714
The 1321-th iter: sec: 0.053516
The 1322-th iter: sec: 0.053625
The 1323-th iter: sec: 0.053532
The 1324-th iter: sec: 0.053602
The 1325-th iter: sec: 0.053419
The 1326-th iter: sec: 0.053695
The 1327-th iter: sec: 0.053488
The 1328-th iter: sec: 0.049138
The 1329-th iter: sec: 0.053139
The 1330-th iter: sec: 0.053251
The 1331-th iter: sec: 0.053558
The 1332-th iter: sec: 0.053966
The 1333-th iter: sec: 0.053684
The 1334-th iter: sec: 0.053595
The 1335-th iter: sec: 0.053621
The 1336-th iter: sec: 0.053382
The 1337-th iter: sec: 0.053712
The 1338-th iter: sec: 0.053436
The 1339-th iter: sec: 0.053948
The 1340-th iter: sec: 0.053559
The 1341-th iter: sec: 0.053429
The 1342-th iter: sec: 0.05391
The 1343-th iter: sec: 0.053283
The 1344-th iter: sec: 0.053877
The 1345-th iter: sec: 0.048916
The 1346-th iter: sec: 0.05334
The 1347-t

The 1576-th iter: sec: 0.055021
The 1577-th iter: sec: 0.052298
The 1578-th iter: sec: 0.053209
The 1579-th iter: sec: 0.053738
The 1580-th iter: sec: 0.053932
The 1581-th iter: sec: 0.05365
The 1582-th iter: sec: 0.053618
The 1583-th iter: sec: 0.053982
The 1584-th iter: sec: 0.054003
The 1585-th iter: sec: 0.053399
The 1586-th iter: sec: 0.054063
The 1587-th iter: sec: 0.053178
The 1588-th iter: sec: 0.054145
The 1589-th iter: sec: 0.053419
The 1590-th iter: sec: 0.053613
The 1591-th iter: sec: 0.054204
The 1592-th iter: sec: 0.053445
The 1593-th iter: sec: 0.053617
The 1594-th iter: sec: 0.054317
The 1595-th iter: sec: 0.053829
The 1596-th iter: sec: 0.053827
The 1597-th iter: sec: 0.053897
The 1598-th iter: sec: 0.053341
The 1599-th iter: sec: 0.05321
The 1600-th iter: sec: 0.053909
The 1601-th iter: sec: 0.05386
The 1602-th iter: sec: 0.054136
The 1603-th iter: sec: 0.053801
The 1604-th iter: sec: 0.053689
The 1605-th iter: sec: 0.053926
The 1606-th iter: sec: 0.053751
The 1607-th

The 1836-th iter: sec: 0.054007
The 1837-th iter: sec: 0.053841
The 1838-th iter: sec: 0.053885
The 1839-th iter: sec: 0.053748
The 1840-th iter: sec: 0.053907
The 1841-th iter: sec: 0.053502
The 1842-th iter: sec: 0.054176
The 1843-th iter: sec: 0.054084
The 1844-th iter: sec: 0.049446
The 1845-th iter: sec: 0.053623
The 1846-th iter: sec: 0.053909
The 1847-th iter: sec: 0.054182
The 1848-th iter: sec: 0.053776
The 1849-th iter: sec: 0.053753
The 1850-th iter: sec: 0.054393
The 1851-th iter: sec: 0.053874
The 1852-th iter: sec: 0.054031
The 1853-th iter: sec: 0.053651
The 1854-th iter: sec: 0.049406
The 1855-th iter: sec: 0.053442
The 1856-th iter: sec: 0.054262
The 1857-th iter: sec: 0.053672
The 1858-th iter: sec: 0.053916
The 1859-th iter: sec: 0.053806
The 1860-th iter: sec: 0.054495
The 1861-th iter: sec: 0.053732
The 1862-th iter: sec: 0.049394
The 1863-th iter: sec: 0.05363
The 1864-th iter: sec: 0.054159
The 1865-th iter: sec: 0.053407
The 1866-th iter: sec: 0.054028
The 1867-

The 2095-th iter: sec: 0.08217
The 2096-th iter: sec: 0.082002
The 2097-th iter: sec: 0.079796
The 2098-th iter: sec: 0.079762
The 2099-th iter: sec: 0.081643
The 2100-th iter: sec: 0.081828
The 2101-th iter: sec: 0.081288
The 2102-th iter: sec: 0.080826
The 2103-th iter: sec: 0.079683
The 2104-th iter: sec: 0.082951
The 2105-th iter: sec: 0.079403
The 2106-th iter: sec: 0.082298
The 2107-th iter: sec: 0.082781
The 2108-th iter: sec: 0.082829
The 2109-th iter: sec: 0.079798
The 2110-th iter: sec: 0.080559
The 2111-th iter: sec: 0.082184
The 2112-th iter: sec: 0.081302
The 2113-th iter: sec: 0.082192
The 2114-th iter: sec: 0.081282
The 2115-th iter: sec: 0.083891
The 2116-th iter: sec: 0.083112
The 2117-th iter: sec: 0.082007
The 2118-th iter: sec: 0.081751
The 2119-th iter: sec: 0.079525
The 2120-th iter: sec: 0.081881
The 2121-th iter: sec: 0.080967
The 2122-th iter: sec: 0.082634
The 2123-th iter: sec: 0.081837
The 2124-th iter: sec: 0.082001
The 2125-th iter: sec: 0.082664
The 2126-

The 2352-th iter: sec: 0.045385
The 2353-th iter: sec: 0.064533
The 2354-th iter: sec: 0.113504
The 2355-th iter: sec: 0.070825
The 2356-th iter: sec: 0.040092
The 2357-th iter: sec: 0.063915
The 2358-th iter: sec: 0.072345
The 2359-th iter: sec: 0.064705
The 2360-th iter: sec: 0.044299
The 2361-th iter: sec: 0.052844
The 2362-th iter: sec: 0.075012
The 2363-th iter: sec: 0.064106
The 2364-th iter: sec: 0.038902
The 2365-th iter: sec: 0.082721
The 2366-th iter: sec: 0.065103
The 2367-th iter: sec: 0.039237
The 2368-th iter: sec: 0.06453
The 2369-th iter: sec: 0.06282
The 2370-th iter: sec: 0.057765
The 2371-th iter: sec: 0.053672
The 2372-th iter: sec: 0.117015
The 2373-th iter: sec: 0.068727
The 2374-th iter: sec: 0.062282
The 2375-th iter: sec: 0.057802
The 2376-th iter: sec: 0.071419
The 2377-th iter: sec: 0.05009
The 2378-th iter: sec: 0.038395
The 2379-th iter: sec: 0.058071
The 2380-th iter: sec: 0.064065
The 2381-th iter: sec: 0.066568
The 2382-th iter: sec: 0.066206
The 2383-th